# 401k Contribution

1. In this example I used Integer Programming, AMPL and Python to optimally allocate contributions to Roth and Traditional.
2. Input: 
    * Roth Annual Max = 5500 (default)
    * Total Annual Contribution = Traditional + Roth = 18500 (default)
3. Ouput:
    * Bi-Weekly dollar amount to contribute to Roth
    * Bi-Weekly dollar amount to contribute to Traditional
    * Surplus 
4. Objective Function: Maximize contribution to Roth, remianing contribute to Traditional. Any amount remaining after contributing Roth and Traditional, assign it to Surplus.  
5. Surplus amount would be something we need to contribute lumpsum in a single paycheck. Surplus is the amount of dollars we could not allocate to Roth and Traditional in \\$1 increments. If we were to split the Surplus equally in each paycheck that would be a fractional dollar (cents). My investment plan does not allow for contributing in cents. The minimum contribution has to be in \\$1 increments. 

In [1]:
#NUMBER_OF_MONTHS = 12
ROTH_MAX = 6000
MAX_CONTRIBUTION = 19500 #Max 401k contribution
NO_OF_PAY_PERIODS_IN_A_YEAR = 26
ROTH_CONTRIBUTION_SO_FAR = 0.0
TRADITIONAL_CONTRIBUTION_SO_FAR = 0.0
NO_OF_PAY_PERIODS_CONTRIBUTED = 0
REMAINING_PAY_PERIODS=NO_OF_PAY_PERIODS_IN_A_YEAR-NO_OF_PAY_PERIODS_CONTRIBUTED
TOTAL_CONTRIBUTION_SO_FAR=ROTH_CONTRIBUTION_SO_FAR+TRADITIONAL_CONTRIBUTION_SO_FAR

In [2]:
model = """
#PART 1: DECISION VARIABLES
var roth >= 0 integer;
var traditional >= 0 integer;
var surplus >= 0;

#PART 2: OBJECTIVE FUNCTION
maximize z: roth*{REMAINING_PAY_PERIODS}*4 + traditional*{REMAINING_PAY_PERIODS}*2 + {TOTAL_CONTRIBUTION_SO_FAR} + surplus;
#The second 2 here is because I value roth twice more than traditional. 

#PART 3: CONSTRAINTS
s.t. M1: roth*{REMAINING_PAY_PERIODS} + {ROTH_CONTRIBUTION_SO_FAR}  <= {ROTH_MAX};
s.t. M2: roth*{REMAINING_PAY_PERIODS} + traditional*{REMAINING_PAY_PERIODS} + {TOTAL_CONTRIBUTION_SO_FAR} + surplus <= {MAX_CONTRIBUTION};
""".format(
    ROTH_MAX=ROTH_MAX, 
    MAX_CONTRIBUTION=MAX_CONTRIBUTION, 
    REMAINING_PAY_PERIODS=REMAINING_PAY_PERIODS,
    TOTAL_CONTRIBUTION_SO_FAR=TOTAL_CONTRIBUTION_SO_FAR,
    ROTH_CONTRIBUTION_SO_FAR=ROTH_CONTRIBUTION_SO_FAR
)

In [3]:
print(model)


#PART 1: DECISION VARIABLES
var roth >= 0 integer;
var traditional >= 0 integer;
var surplus >= 0;

#PART 2: OBJECTIVE FUNCTION
maximize z: roth*26*4 + traditional*26*2 + 0.0 + surplus;
#The second 2 here is because I value roth twice more than traditional. 

#PART 3: CONSTRAINTS
s.t. M1: roth*26 + 0.0  <= 6000;
s.t. M2: roth*26 + traditional*26 + 0.0 + surplus <= 19500;



In [6]:
from amplpy import AMPL, Environment

ampl = AMPL(Environment('/media/second_drive/PERSONAL_PROJECTS/AMPLprojects/app-docker/amplide.linux64'))
print(ampl.getOption('solver'))
ampl.setOption('solver','cplex')
print(ampl.getOption('solver'))
#ampl.read('example.mod') #read the model
ampl.eval(model)
ampl.solve()
#ampl.eval('objective z; solve;')
#ampl.display('z', 'roth', 'traditional', 'surplus')

minos
cplex
CPLEX 12.9.0.0: optimal integer solution; objective 50960
0 MIP simplex iterations
0 branch-and-bound nodes


In [7]:
z=ampl.getValue('z')
roth=ampl.getValue('roth')
traditional=ampl.getValue('traditional')
surplus=ampl.getValue('surplus')

In [8]:
print(z)
print(roth)
print(traditional)
print(surplus)

50960.0
230.0
520.0
0.0


In [9]:
for const in ampl.getConstraints():
    print(const[1].get())
    print(const[1].getValues())

subject to M1:
	26*roth <= 6000;
  M1.dual   
     0      

subject to M2:
	26*roth + 26*traditional + surplus <= 19500;
  M2.dual   
     1      



In [10]:
annual_roth_amt = roth*REMAINING_PAY_PERIODS
annual_traditional_amt = traditional*REMAINING_PAY_PERIODS
pstring = """
objective function                 = {z}
bi-weekly roth contribution        = ${roth}
bi-weekly traditional contribution = ${traditional}
roth contribution                  = ${rothContribution}
traditional contribution           = ${traditionalContribution}
total (without surplus)            = ${total}
total (with surplus)               = ${totalS}
"""


print(pstring.format(z=z,
                     roth=roth,
                     traditional=traditional,
                     rothContribution=annual_roth_amt+ROTH_CONTRIBUTION_SO_FAR,
                     traditionalContribution=annual_traditional_amt+TRADITIONAL_CONTRIBUTION_SO_FAR,
                     total=annual_roth_amt+annual_traditional_amt+TOTAL_CONTRIBUTION_SO_FAR,
                     totalS=annual_roth_amt+annual_traditional_amt+TOTAL_CONTRIBUTION_SO_FAR+surplus
                    ))


objective function                 = 50960.0
bi-weekly roth contribution        = $230.0
bi-weekly traditional contribution = $520.0
roth contribution                  = $5980.0
traditional contribution           = $13520.0
total (without surplus)            = $19500.0
total (with surplus)               = $19500.0



In [11]:
variables = list()
for variable in ampl.getVariables():
        #print(variable[1].get())
        variables.append((variable[0],
                          variable[1].lb(),
                          variable[1].value(),
                          variable[1].ub(),
                          variable[1].rc()))

In [12]:
ampl.eval('display _nvars, _ncons;')

_nvars = 3
_ncons = 2



In [13]:
import pandas as pd
df = pd.DataFrame(variables, columns=['Variable',
                                      'LowerBound',
                                      'Value',
                                      'UpperBound',
                                      'ReducedCost']).set_index('Variable')

In [14]:
df

,LowerBound,Value,UpperBound,ReducedCost
Variable,,,,
roth,0.0,230.0,230.0,78.0
surplus,0.0,0.0,inf,0.0
traditional,0.0,520.0,750.0,26.0
